## Installing Docker

To install Docker, follow the instructions on the official Docker website: [Docker Installation Guide](https://docs.docker.com/get-docker/).

### Building the Docker Image
First make sure Docker is running. 

```bash
docker build -t <image_name> -f <path_where_dockerfile_is_located>\Dockerfile <lerobot_dir>
```
For example:

```bash
docker build -t lerobot_img -f docker\lerobot-gpu\Dockerfile C:\Users\jprij\Documents\lerobot
```

## Compatibility with Windows WSL2

Assuming
- Graphics driver already installed
- Windows 11
- WSL is already installed (automatic with windows 11)
If not, follow the instructions on the official Microsoft website: [WSL Installation Guide](https://docs.microsoft.com/en-us/windows/wsl/install).

Follow step 3.Option1 from the Nvidia [website](https://docs.nvidia.com/cuda/wsl-user-guide/index.html)
Select the Ubuntu-wsl option and run the commands according to Nvidia's instructions.

Check if the installation works, by running:
```bash
docker run --rm --gpus all nvidia/cuda:11.8.0-base-ubuntu22.04 nvidia-smi
```

this generates a docker container based on an Nvidia cuda image, runs Nvidia-smi, and then discards the container. 

The '--gpus all' option should be added because it tells docker to use the gpu.



## Run a container from the image

```bash
docker run -it --gpus all --shm-size=8g --name <container_name> <image_name> 
```

Validate by running the following python code inside the container:
```python
import torch
import numpy as np
torch.cuda.is_available() # Should return 'True'

device = torch.device('cuda')
a = np.array([[1,2],[2,3]])
b = torch.from_numpy(a)
b.to(device) # Should return a cuda tensor
print(b)
```

## Train a policy

First log in to Hugginface and WandB

```Python
HF_USER=$(huggingface-cli whoami | head -n 1)
```

```python
import wandb
wandb.login(key='WB_KEY')
```

Then run the training script, for example: 


In [ ]:
HF_KEY = # Your Hugging Face API key
WB_KEY = # Your Weights & Biases API key

!huggingface-cli login --token HF_KEY

HF_USER=$(huggingface-cli whoami | head -n 1)

import wandb
wandb.login(key='WB_KEY')

In [ ]:
# Run a training job with the specified parameters
!python lerobot/scripts/train.py --dataset.repo_id=${HF_USER}/record-test-5 --policy.type=act --output_dir=outputs/train/act_so101_test_1 --job_name=act_so101_test_1 --policy.device=cuda --wandb.enable=true